#### Feature creation based on inquiry table

In [5]:
import sys, os, json
import pandas as pd
from sqlalchemy import create_engine
import sys;
import xml.dom.minidom;
import json
# import jmespath
import time
# import duckdb
import numpy as np
import re

sys.path.insert(1, "../../src")
import exp_inq_attr_builder as exp_inq

%load_ext autoreload
%autoreload 2

#### create flag based on raw table

In [6]:
sql_base="""
with a as (select *,
                  case
                      when (KOBcode in ('AC', 'AF', 'AL', 'AN', 'AU', 'AZ', 'FA', 'ZA') and TypeCode not in
                                                                                        ('04', '05', '08', '0C', '16',
                                                                                         '17', '19', '25', '26', '27',
                                                                                         '29', '2C', '48', '4F', '5A',
                                                                                         '5B', '6B', '6D', '85', '87',
                                                                                         '89', '9A')) or
                           (TypeCode in ('00', '13', '3A') and
                            KOBcode not in
                            ('BM', 'FB', 'FL', 'FM', 'FR', 'RE', 'YA', 'YC', 'YL', 'YZ', 'ZB', 'ZC', 'ZF', 'ZY'))
                          then 'auto'
                      when (KOBcode in ('BB', 'BC', 'BN', 'BO', 'BS', 'FC', 'FS', 'NF', 'NS', 'NU', 'OC', 'OZ') and
                            TypeCode not in
                            ('00', '04', '05', '08', '0C', '13', '16', '17', '19', '25', '26', '27', '29', '2C', '3A',
                             '48', '4F', '5A', '5B', '6B', '6D', '85', '87', '89', '9A'))
                          or (TypeCode in ('07', '18', '37', '0G', '2A') and
                              KOBcode not in
                              ('AC', 'AF', 'AL', 'AN', 'AU', 'AZ', 'BM', 'FA', 'FB', 'FL', 'FM', 'FR', 'RE', 'YA', 'YC',
                               'YL', 'YZ', 'ZB', 'ZC', 'ZA', 'ZF', 'ZY')) then 'bank'
                      when ((substring(KOBcode, 1, 1) similar to 'C|D|G|H|J|L|S|T') or
                            KOBcode in ('AP', 'AT', 'MP', 'ND', 'PF', 'QZ', 'ZR')) and TypeCode not in
                                                                                   ('00', '04', '05', '08', '0C', '13',
                                                                                    '16', '17', '19', '25', '26', '27',
                                                                                    '29', '2C', '3A', '48', '4F', '5A',
                                                                                    '5B', '6B', '6D', '85', '87', '9A')
                          then 'retail'
                      when KOBcode = 'FP' and TypeCode not in
                                          ('00', '04', '05', '07', '08', '0C', '0G', '13', '16', '17', '18', '19', '25',
                                           '26', '27', '29', '2A', '2C', '37', '3A', '48', '4F', '5A', '5B', '6B', '6D',
                                           '85', '87', '89', '9A') then 'personal_finance'
                      when TypeCode = '29' and KOBcode not in
                                           ('AC', 'AF', 'AL', 'AN', 'AP', 'AT', 'AU', 'AZ', 'BB', 'BC', 'BM', 'BN',
                                            'BO', 'BS', 'FA', 'FB', 'FC', 'FL', 'FM', 'FP', 'FR', 'FS', 'MP', 'ND',
                                            'NF', 'NS', 'NU', 'OC', 'OZ', 'PF', 'QZ', 'RE', 'YA', 'YC', 'YL', 'YZ',
                                            'ZA', 'ZB', 'ZC', 'ZF', 'ZR', 'ZY')
                          and KOBcode not in ('C', 'D', 'G', 'H', 'J', 'L', 'S', 'T') then 'real_estate'
                      else 'others' end                                   as product_type,
                (cast(credit_pull_date as date)-cast(Date as date))                     as inq_date_dif                
           from inquiry_df)
"""

#### define input

In [7]:
# define parameter config file
param_config = {
    'descriptor':['num_inq'],
    'source_table':['inquiry'],
    'loan_type':['','personal_finance'],
    'date_window':['','7day','14day','1month','3month','6month','12month'],
    'sublist':['',"('LENDING CLUB/WEBBANK','PROSPER/WEBBANK','ONEMAIN','ONEMAIN FINANCIAL SERV','FREEDOM PLUS/CROSS RIV','LENDINGPOINT/FINWISE B','FIRST CENTER/BEST EGG','ONEMAIN FINANCIAL GROU','COMENITYCB/LENDINGCLUB','LENDINGPOINT LLC')"]
}

#### generate parameter and sql code

In [8]:
#  generate list of parameters based on parameter config file
param_list = []
inq_dict = pd.DataFrame()
for i in range(0,len(param_config['descriptor'])):
        for j in range(0,len(param_config['source_table'])):
                for k in range(0, len(param_config['loan_type'])):
                        for l in range(0, len(param_config['date_window'])):
                                for m in range(0, len(param_config['sublist'])):
                                    # define descriptor
                                    descriptor = param_config['descriptor'][i]
                                    # define source_table
                                    source_table = param_config['source_table'][j]
                                    # define loan_type
                                    loan_type = param_config['loan_type'][k]
                                    # define window
                                    try:
                                        window = re.findall(r'\d+', param_config['date_window'][l])[0]
                                    except:
                                        window = ''
                                    # define unit for window
                                    try:
                                        window_unit = re.findall("[a-zA-Z]+", param_config['date_window'][l])[0]
                                    except:
                                        window_unit = ''
                                    # define sublist
                                    sublist = param_config['sublist'][m]
                                    # define parameters
                                    param_new = {'descriptor': descriptor,
                                                'source_table': source_table,
                                                'loan_type': loan_type,
                                                'window': window,
                                                'window_unit': window_unit,
                                                'sublist': sublist}
                                    attr_0 = exp_inq.exp_inq_attr_builder(param_new)
                                    # field_name 
                                    field_name = attr_0.get_attr_name()
                                    # long description
                                    long_description =  attr_0.get_attr_name(1)
                                    # generate list of parameters based on parameter config file
                                    param_new.update({'field_name': field_name,
                                                'long_description': long_description})
                                    inq_dict_new = pd.DataFrame({'field_name':[field_name],'long_description':[long_description]})
                                    if param_new not in param_list:
                                        param_list.append(param_new)
                                        inq_dict = inq_dict.append(inq_dict_new)

C:\Users\rfrancis\AppData\Local\Temp\ipykernel_14172\2935697514.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  inq_dict = inq_dict.append(inq_dict_new)
C:\Users\rfrancis\AppData\Local\Temp\ipykernel_14172\2935697514.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  inq_dict = inq_dict.append(inq_dict_new)
C:\Users\rfrancis\AppData\Local\Temp\ipykernel_14172\2935697514.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  inq_dict = inq_dict.append(inq_dict_new)
C:\Users\rfrancis\AppData\Local\Temp\ipykernel_14172\2935697514.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  inq_dict = inq_dict.append(inq_dict_new)
C:\Users\rfr

In [ ]:
# inq_dict.to_csv('inq_dict.csv')

In [9]:
# generate sql code for aggregation section
sql_agg = ""
for params_inq  in param_list:
    attr = exp_inq.exp_inq_attr_builder(params_inq)
    sql_agg = sql_agg +","  + attr.get_inq_stats_sql()    

In [11]:
# generate sql code: replace placeholder with sql_agg
sql = sql_base + """
select id, applicant_type, credit_pull_id
--<placeholder>--
from a
group by 1, 2, 3
"""
sql = sql.replace("--<placeholder>--", sql_agg)

In [13]:
print(sql)


with a as (select *,
                  case
                      when (KOBcode in ('AC', 'AF', 'AL', 'AN', 'AU', 'AZ', 'FA', 'ZA') and TypeCode not in
                                                                                        ('04', '05', '08', '0C', '16',
                                                                                         '17', '19', '25', '26', '27',
                                                                                         '29', '2C', '48', '4F', '5A',
                                                                                         '5B', '6B', '6D', '85', '87',
                                                                                         '89', '9A')) or
                           (TypeCode in ('00', '13', '3A') and
                            KOBcode not in
                            ('BM', 'FB', 'FL', 'FM', 'FR', 'RE', 'YA', 'YC', 'YL', 'YZ', 'ZB', 'ZC', 'ZF', 'ZY'))
                          then 'auto'
        

##### loop through each| quarter and generate inquiry attributes

In [7]:
quarter_list = ['2021Q2', '2021Q3', '2021Q4', '2022Q1']

In [8]:
inquiry_attr = pd.DataFrame()

for quarter in quarter_list:
    print(quarter)
    inquiry_df = pd.read_parquet(f's3://sofi-data-science/hpeng/pl-gen4/data_dump/raw_tables/feature_creation/inquiry_{quarter}.parquet')
    inquiry_attr_new = duckdb.query(sql).to_df()
    inquiry_attr = inquiry_attr.append(inquiry_attr_new)

# dedup here, one credit pull might be linked to two id; if two id are in different quarter, there will be duplication
inquiry_attr = inquiry_attr.drop_duplicates()
inquiry_attr.to_parquet(f's3://sofi-data-science/hpeng/pl-gen4/data_dump/raw_tables/feature_creation/agg_inquiry_attr_2021Q1_202201.parquet')

2021Q2
2021Q3
2021Q4
2022Q1


In [9]:
inquiry_attr.shape

(1908650, 31)

In [10]:
inquiry_attr.head()

,id,applicant_type,credit_pull_id,sofi_num_inq,sofi_num_inq_list,sofi_num_inq_7day,sofi_num_inq_list_7day,sofi_num_inq_14day,sofi_num_inq_list_14day,sofi_num_inq_1month,...,sofi_num_inq_personal_finance_14day,sofi_num_inq_list_personal_finance_14day,sofi_num_inq_personal_finance_1month,sofi_num_inq_list_personal_finance_1month,sofi_num_inq_personal_finance_3month,sofi_num_inq_list_personal_finance_3month,sofi_num_inq_personal_finance_6month,sofi_num_inq_list_personal_finance_6month,sofi_num_inq_personal_finance_12month,sofi_num_inq_list_personal_finance_12month
0,13963467,PRIMARY,18772624,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,13963481,PRIMARY,18772665,14.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,13963484,PRIMARY,18772726,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13963549,PRIMARY,18772775,6.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,13963629,PRIMARY,18772927,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
